In [1]:
#Nos quitamos los warning...
import warnings
warnings.filterwarnings('ignore')
#importamos las librerias y recursos
import time
import pandas as pd 
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn import utils
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans

In [2]:
#Declaramos la funcion de clasificación de paquetes (clean,scanner,....)

def read_dataframe(file_csv):
    file_dir = file_csv
    df = pd.read_csv(file_dir,index_col="No.",encoding="utf-8",delimiter=";")
    df["type_packet"] = file_dir[:-4]
    clean_dataframe(df)

    return df
def read_dataframe2(file_csv):
    file_dir = file_csv
    df = pd.read_csv(file_dir,index_col="No.",encoding="ISO-8859-1",delimiter=",")
    df["type_packet"] = file_dir[:-4]
    clean_dataframe(df)

    return df
#Declaramos la funcion de limpieza de valores erroneos(NaN) Fill NaN, Criterio: Number = -1, String = ""
def clean_dataframe(df):
    for feature in df.columns:
        if df[feature].dtype == "object" or df[feature].dtype == "category":
            df[feature].fillna("",inplace=True)
            if feature == "S7_Message":
                    df["S7_Message"] = [str(bytearray.fromhex(str(x)).decode()) if x != "" else "" for x in df[feature]]
            if feature == "http_data":
                 df["http_data"] = [True if x !="" else False for x in df[feature] ]
            if feature == "http_user":
                 df["http_user"] = [True if x !="" else False for x in df[feature] ]    
        else:
            df[feature].fillna(-1,inplace=True)
    df.drop(["Source","Destination","Message_String","S7_Message","Mod_Data","Time","Info"],axis=1,inplace=True)
def reduce_dataframes(final_train,final_test):
    ##Let's try to reconvert object-types to primitive type
    final_train = final_train.infer_objects()
    final_test = final_test.infer_objects()
    ###Now let's downcast types to reduce memory usage 
    fcols = final_train.select_dtypes('float').columns
    icols = final_train.select_dtypes('integer').columns
    ###Train dataset downcast 
    final_train[fcols] = final_train[fcols].apply(pd.to_numeric, downcast='float')
    final_train[icols] = final_train[icols].apply(pd.to_numeric, downcast='integer')
    ###Test dataset downcast
    final_test[fcols] = final_test[fcols].apply(pd.to_numeric, downcast='float')
    final_test[icols] = final_test[icols].apply(pd.to_numeric, downcast='integer')
    return final_train,final_test


In [3]:
df_tcpfl = read_dataframe2("brute_http.csv")
df_tcpfl.drop(df_tcpfl[(df_tcpfl.Protocol == "BEEP")| (df_tcpfl.Protocol == "H1") | (df_tcpfl.Protocol == "IRC") |
                       (df_tcpfl.Protocol == "R3") | (df_tcpfl.Protocol == "X11")].index,inplace=True)
print(df_tcpfl.type_packet)
#TCP SCANNER GROUP
df_s_ack = read_dataframe("scanner_ack.csv")
df_s_fin = read_dataframe("scanner_fin.csv")
df_s_null= read_dataframe("scanner_null.csv")
df_s_tcp = read_dataframe("scanner_tcp.csv")
df_s_xmas= read_dataframe("scanner_xmas.csv")

df_scan = pd.concat([df_s_ack,df_s_fin,df_s_null,df_s_tcp,df_s_xmas],axis=0)
df_scan.type_packet= "scanning"

No.
67       brute_http
68       brute_http
69       brute_http
70       brute_http
73       brute_http
            ...    
23987    brute_http
23991    brute_http
23994    brute_http
23995    brute_http
24000    brute_http
Name: type_packet, Length: 4170, dtype: object


In [4]:
df_emulator = pd.concat([df_scan,df_tcpfl],axis=0)
df_emulator.drop(["Protocol","Delay (tcp)","Delay"],axis=1,inplace=True)
df_emulator.columns

Index(['srcPort', 'dstPort', 'Length', 'tcp_Flags', 'tcp_WindowSize',
       'OPC_Message_Size', 'OPC_Sec.Requ_ID', 'MessageType', 'S7_ROSCTR',
       'S7_Data_Length', 'S7_Function', 'S7_Error_Code', 'Mod_Length',
       'Mod_Transa_ID', 'Mod_Function', 'Mod_Req_Frame', 'Mod_Req_Delay',
       'http_user', 'http_data', 'type_packet'],
      dtype='object')

In [5]:
df_scanning_dataset = read_dataframe2("kaggle_scanning.csv")
df_scanning_dataset.type_packet = "scanning"
df_dos_dataset = read_dataframe2("kaggle_brute.csv")
df_dos_dataset.type_packet = "brute_http"
df_dataset = pd.concat([df_scanning_dataset,df_dos_dataset],axis=0)
print(df_dataset.columns)
df_dataset.drop(["Protocol","icmp_data"],axis=1,inplace=True)
display(df_dataset)

Index(['Protocol', 'Length', 'OPC_Message_Size', 'OPC_Sec.Requ_ID',
       'MessageType', 'S7_ROSCTR', 'S7_Data_Length', 'S7_Function',
       'S7_Error_Code', 'Mod_Length', 'Mod_Transa_ID', 'Mod_Function',
       'Mod_Req_Frame', 'Mod_Req_Delay', 'tcp_Flags', 'tcp_WindowSize',
       'http_data', 'http_user', 'icmp_data', 'type_packet'],
      dtype='object')


,Length,OPC_Message_Size,OPC_Sec.Requ_ID,MessageType,S7_ROSCTR,S7_Data_Length,S7_Function,S7_Error_Code,Mod_Length,Mod_Transa_ID,Mod_Function,Mod_Req_Frame,Mod_Req_Delay,tcp_Flags,tcp_WindowSize,http_data,http_user,type_packet
No.,,,,,,,,,,,,,,,,,,
1,60,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0x002,512.0,-1.0,-1.0,scanning
2,54,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0x014,0.0,-1.0,-1.0,scanning
3,60,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0x002,512.0,-1.0,-1.0,scanning
4,54,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0x014,0.0,-1.0,-1.0,scanning
5,60,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0x002,512.0,-1.0,-1.0,scanning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053889,66,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0x010,65152.0,0.0,0.0,brute_http
1053890,66,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0x010,63488.0,0.0,0.0,brute_http
1053891,66,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0x010,63488.0,0.0,0.0,brute_http


In [7]:
X = df_dataset.drop("type_packet",axis=1)
X = pd.get_dummies(X)
y = df_dataset.type_packet
print(display(X))
X_test = df_emulator.drop("type_packet",axis=1)
y_test = df_emulator.type_packet
X_test = pd.get_dummies(X_test)


#Features not found in our data
X_test["tcp_Flags_0x011"] = -1
X_test["tcp_Flags_0x012"] = -1
X_test["tcp_Flags_0x014"] = -1
X_test["tcp_Flags_0x019"] = -1
X_test = X_test[X.columns]
print(display(X_test))
model = DecisionTreeClassifier()


model.fit(X,y)

pred = model.predict(X_test)

print(classification_report(y_test,pred))

,Length,OPC_Message_Size,OPC_Sec.Requ_ID,MessageType,S7_ROSCTR,S7_Data_Length,S7_Function,S7_Error_Code,Mod_Length,Mod_Transa_ID,...,http_user,tcp_Flags_,tcp_Flags_0x002,tcp_Flags_0x004,tcp_Flags_0x010,tcp_Flags_0x011,tcp_Flags_0x012,tcp_Flags_0x014,tcp_Flags_0x018,tcp_Flags_0x019
No.,,,,,,,,,,,,,,,,,,,,,
1,60,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,False,True,False,False,False,False,False,False,False
2,54,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,False,False,False,False,False,False,True,False,False
3,60,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,False,True,False,False,False,False,False,False,False
4,54,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,False,False,False,False,False,False,True,False,False
5,60,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053889,66,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.0,False,False,False,True,False,False,False,False,False
1053890,66,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.0,False,False,False,True,False,False,False,False,False
1053891,66,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.0,False,False,False,True,False,False,False,False,False


None


,Length,OPC_Message_Size,OPC_Sec.Requ_ID,MessageType,S7_ROSCTR,S7_Data_Length,S7_Function,S7_Error_Code,Mod_Length,Mod_Transa_ID,...,http_user,tcp_Flags_,tcp_Flags_0x002,tcp_Flags_0x004,tcp_Flags_0x010,tcp_Flags_0x011,tcp_Flags_0x012,tcp_Flags_0x014,tcp_Flags_0x018,tcp_Flags_0x019
No.,,,,,,,,,,,,,,,,,,,,,
1,54,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,False,False,False,True,-1,-1,-1,False,-1
2,54,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,False,False,False,True,-1,-1,-1,False,-1
3,54,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,False,False,False,True,-1,-1,-1,False,-1
4,54,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,False,False,False,True,-1,-1,-1,False,-1
5,54,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,False,False,False,True,-1,-1,-1,False,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23987,250,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.0,False,False,False,False,-1,-1,-1,True,-1
23991,76,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.0,False,False,False,False,-1,-1,-1,False,-1
23994,143,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.0,False,False,False,False,-1,-1,-1,True,-1


None
              precision    recall  f1-score   support

  brute_http       1.00      1.00      1.00      4170
    scanning       1.00      1.00      1.00      5373

    accuracy                           1.00      9543
   macro avg       1.00      1.00      1.00      9543
weighted avg       1.00      1.00      1.00      9543

